# Topic modelling with Gensim : the LDA algorithm

LDA is a generative statistical model that allows sets of observations to be explained by unobserved groups that explain why some parts of the data are similar.

### 1. Loading of the required libraries and vocabularies.
***



In [160]:
import gensim
import pandas as pd

import spacy
import sys

## Run once this chunk with the command below active. Then comment it out and run the notebook
!{sys.executable} -m spacy download en

nlp = spacy.load("en_core_web_sm", disable=['parser', 'ner'])

import nltk
import re
import pprint
import os
import sys


from gensim import corpora

import nltk
from nltk import sent_tokenize
from nltk.corpus import stopwords

##nltk.download('stopwords')
##stop_words = stopwords.words('english')

import pyLDAvis
import pyLDAvis.gensim_models as gensimvis # don't skip this
import matplotlib.pyplot as plt
%matplotlib inline

import logging
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.ERROR)

import warnings
warnings.filterwarnings("ignore",category=DeprecationWarning)

from pprint import pprint



[!] As of spaCy v3.0, shortcuts like 'en' are deprecated. Pleaseuse the full
pipeline package name 'en_core_web_sm' instead.
[+] Download and installation successful
You can now load the package via spacy.load('en_core_web_sm')


### Step 2. Pre-processing

* Read two files exported from the database which contain the SE Glossary articles definitions and their titles.
* In later versions, the **corresponding tables will be directly exported from the database**.
* Merge by_id_ and discard records with duplicate titles and/or definitions.


In [161]:
dat1= pd.read_csv("ESTAT_dat_concepts_2021_04_08.csv",sep=";")
dat2= pd.read_csv("ESTAT_dat_link_info_2021_04_08.csv",sep=";")
Gloss_concepts = pd.merge(dat1,dat2,on=['id'])
del(dat1,dat2)

Gloss_concepts = Gloss_concepts[['id','title','definition']]

Gloss_concepts = Gloss_concepts.drop_duplicates(subset=["definition"])
Gloss_concepts = Gloss_concepts.dropna(axis=0,subset=["definition"])
Gloss_concepts = Gloss_concepts.drop_duplicates(subset=["title"])
Gloss_concepts = Gloss_concepts.dropna(axis=0,subset=["title"])

Gloss_concepts.reset_index(drop=True, inplace=True)
Gloss_concepts

,id,title,definition
0,1,Arrival of tourist at a tourist accommodation ...,See arrival disambiguation page fo...
1,3,Nights spent,A night spent or tourism night ...
2,4,Supply side tourism statistics,Regulation (EU) No 692/2011 of 6 July 2011 e...
3,5,Tourist,"For tourism statistics, a travell..."
4,7,Asia-Europe Meeting (ASEM),"The Asia-Europe Meeting , abbreviated..."
...,...,...,...
1279,2879,Statistical territory,The statistical territory of the ...
1280,2881,Statistical indicator,A statistical indicator is the repres...
1281,2885,Tourism industries,For the article Tourism industries - eco...
1282,2891,Sawnwood,Sawnwood is wood that has been produced ...


### 3. Pre-processing input data (cont).
***

Next we tokenize the texts - definitions in the articles, select tokens with minimum length 5, delete stop words and apply a simple pre-processing (convert to lowercase, drop accents). 

The result, _texts_ is a list with 1286 elements corresponding to the records in the dataframe _df_.

In [162]:
#Tokenize texts and clean-up text.
from gensim.parsing.preprocessing import remove_stopwords
def sent_to_words(sentences):
    for sentence in sentences:
        sentence = remove_stopwords(sentence) ## remove stop words
        tokens = gensim.utils.tokenize(sentence)
        sentence = [token for token in tokens if len(token) >= 5] ##minimum length = 5 
        yield(gensim.utils.simple_preprocess(str(sentence), deacc=True))  # to lower + deacc=True removes punctuations
        
texts = list(sent_to_words(Gloss_concepts['definition']))
print('\nFirst 10 texts: \n',texts[:10])
print('\nTotal texts: ',len(texts),'\n')



First 10 texts: 
 [['arrival', 'disambiguation', 'meanings', 'arrival', 'within', 'context', 'european', 'union', 'tourism', 'statistics', 'arrival', 'defined', 'tourist', 'arrives', 'tourist', 'accommodation', 'establishment', 'rented', 'accommodation', 'rented', 'accommodation', 'scope', 'tourism', 'regulation', 'however', 'collected', 'tourist', 'resident', 'resident', 'country', 'establishment', 'located', 'limit', 'applied', 'children', 'counted', 'adults', 'overnight', 'stays', 'children', 'charge'], ['night', 'spent', 'tourism', 'night', 'overnight', 'night', 'guest', 'tourist', 'resident', 'resident', 'actually', 'spends', 'sleeps', 'stays', 'tourist', 'accommodation', 'establishment', 'rented', 'accommodation'], ['regulation', 'establishes', 'common', 'framework', 'systematic', 'development', 'production', 'dissemination', 'european', 'statistics', 'tourism', 'purpose', 'member', 'states', 'transmit', 'harmonised', 'statistics', 'tourism', 'supply', 'demand', 'tourism', 'supp

### 4. Creation of corpus and terms frequencies.
***

Next we create:
* a dictionary from _texts_ with name _id2word_. This has 7258 unique tokens. 
* a mapping with name _corpus_ of the texts into lists with tuples: (word id, frequency in each text)


In [163]:
#Create Dictionary
id2word = corpora.Dictionary(texts) #Gensim creates a unique id for each word in the document. 
print(id2word,'\n')

## The produced corpus shown above is a mapping of (word_id, word_frequency).

#Alternatively:
corpus = [id2word.doc2bow(text) for text in texts] #corpus package automatically creates a set of corpus reader instances that can be used to access the corpora in the NLTK data package.

print('First 10 texts:\n')
print(corpus[:10])
print('\nTotal texts: ',len(corpus))

#Human readable format of corpus (term-frequency)
#[[(id2word[id], freq) for id, freq in cp] for cp in corpus[:1]]

Dictionary(7304 unique tokens: ['accommodation', 'adults', 'applied', 'arrival', 'arrives']...) 

First 10 texts:

[[(0, 3), (1, 1), (2, 1), (3, 3), (4, 1), (5, 1), (6, 2), (7, 1), (8, 1), (9, 1), (10, 1), (11, 1), (12, 1), (13, 2), (14, 1), (15, 1), (16, 1), (17, 1), (18, 1), (19, 1), (20, 1), (21, 2), (22, 2), (23, 1), (24, 1), (25, 1), (26, 2), (27, 3), (28, 1), (29, 1)], [(0, 2), (13, 1), (19, 1), (21, 1), (22, 2), (25, 1), (26, 1), (27, 2), (30, 1), (31, 1), (32, 3), (33, 1), (34, 1), (35, 1)], [(0, 3), (10, 4), (14, 1), (20, 1), (21, 1), (22, 2), (24, 5), (26, 6), (27, 1), (36, 1), (37, 2), (38, 1), (39, 2), (40, 1), (41, 1), (42, 1), (43, 1), (44, 1), (45, 1), (46, 1), (47, 1), (48, 1), (49, 1), (50, 1), (51, 1), (52, 1), (53, 1), (54, 4), (55, 3), (56, 1), (57, 2), (58, 1), (59, 1), (60, 1), (61, 1)], [(10, 1), (19, 3), (22, 1), (24, 1), (26, 2), (27, 1), (52, 3), (60, 2), (62, 1), (63, 1), (64, 1), (65, 1), (66, 1), (67, 1), (68, 1), (69, 1), (70, 1), (71, 1), (72, 1), (73, 1)

### 5. Dominant topic in each document.
***
Αt this stage we build the  _lda_model_, through which the dominant topic for each document will be extracted and we show the weight of the topic and the keywords. We define a dataframe _df_topic_sents_keywords_  in order to store these dominant topics. 

In function _format_topics_sentences()_ below, the list _ldamodel[corpus]_ has one nested list element per text. Each nested list contains tuples (topic, contribution). We sort each nested list by descending contribution to find the dominant topic and then, we retrieve for this topic, the list _wp_ of tuples (word, probability) for the most probable words, using the function _ldamodel.show_topic()_. We join these words into a list and put the result in column 'Topic_Keywords' of the dataframe.


In [164]:
#What is the Dominant topic and its percentage contribution in each document.

def format_topics_sentences(ldamodel=None, corpus=corpus):
    # Init output
    sent_topics_df = pd.DataFrame()

    # Get main topic in each document
    for i, row_list in enumerate(ldamodel[corpus]): 
        row = row_list[0] if ldamodel.per_word_topics else row_list ## ldamodel.per_word_topics is False          
        #print(row)
        row = sorted(row, key=lambda x: (x[1]), reverse=True) ## sort the nested list by descending contribution
        # Get the Dominant topic, Perc Contribution and Keywords for each document
        topic_num, prop_topic = row[0]
        wp = ldamodel.show_topic(topic_num)
        topic_keywords = ", ".join([word for word, prop in wp])
        sent_topics_df = sent_topics_df.append(pd.Series([int(topic_num), round(prop_topic,4), topic_keywords]), ignore_index=True)
    sent_topics_df.columns = ['Dominant_Topic', 'Perc_Contribution', 'Topic_Keywords']

    # Add original tokenized text and title to the end of the output
    contents = pd.Series(texts)
    sent_topics_df = pd.concat([sent_topics_df, contents], axis=1)
    sent_topics_df = pd.concat([sent_topics_df, Gloss_concepts['id'],Gloss_concepts['title']], axis=1)
    return(sent_topics_df)

In [165]:
#Build LDA model
lda_model = gensim.models.ldamodel.LdaModel(corpus=corpus,
                                           id2word=id2word,
                                           num_topics=20, 
                                           random_state=100,
                                           update_every=1,
                                           chunksize=100,
                                           passes=10,
                                           alpha='auto',
                                           per_word_topics=True)

In [166]:
df_topic_sents_keywords = format_topics_sentences(ldamodel=lda_model, corpus=corpus)
df_topic_sents_keywords.rename(columns = {0:'Text tokenized definition'}, inplace = True)
df_topic_sents_keywords.rename(columns = {'id':'Text id','title':'Text title'}, inplace = True)
df_topic_sents_keywords = df_topic_sents_keywords[['Text id','Text title','Text tokenized definition','Dominant_Topic', 'Perc_Contribution', 'Topic_Keywords']]
df_topic_sents_keywords['Dominant_Topic'] = df_topic_sents_keywords['Dominant_Topic'].astype(int)
df_topic_sents_keywords


,Text id,Text title,Text tokenized definition,Dominant_Topic,Perc_Contribution,Topic_Keywords
0,1,Arrival of tourist at a tourist accommodation ...,"[arrival, disambiguation, meanings, arrival, w...",17,0.3693,"employed, place, stock, private, similar, acco..."
1,3,Nights spent,"[night, spent, tourism, night, overnight, nigh...",17,0.4157,"employed, place, stock, private, similar, acco..."
2,4,Supply side tourism statistics,"[regulation, establishes, common, framework, s...",17,0.5458,"employed, place, stock, private, similar, acco..."
3,5,Tourist,"[tourism, statistics, traveller, moves, differ...",17,0.7203,"employed, place, stock, private, similar, acco..."
4,7,Asia-Europe Meeting (ASEM),"[europe, meeting, abbreviated, informal, proce...",7,0.7317,"european, member, countries, states, union, ec..."
...,...,...,...,...,...,...
1279,2879,Statistical territory,"[statistical, territory, european, union, cont...",3,0.2045,"statistics, level, statistical, international,..."
1280,2881,Statistical indicator,"[statistical, indicator, representation, stati...",3,0.2126,"statistics, level, statistical, international,..."
1281,2885,Tourism industries,"[article, tourism, industries, economic, analy...",10,0.2471,"activities, water, households, energy, prices,..."
1282,2891,Sawnwood,"[sawnwood, produced, sawing, lengthways, profi...",0,0.1992,"activities, activity, market, product, product..."


### 6. Most representative document for each topic.
***

**Need to add the document id**

We want to take samples of documents that best represent a given topic, to have a complete analysis.This code receives the most exemplary document for each topic. We create the _sent_topics_sorteddf_mallet_ and these Topic Keywords are presented below.

In [167]:
# Group top 5 sentences under each topic
sent_topics_sorteddf_mallet = pd.DataFrame()

sent_topics_outdf_grpd = df_topic_sents_keywords[['Text id','Text title','Dominant_Topic', 'Perc_Contribution', 'Topic_Keywords']].groupby('Dominant_Topic')

for i, grp in sent_topics_outdf_grpd:
    sent_topics_sorteddf_mallet = pd.concat([sent_topics_sorteddf_mallet, 
                                             grp.sort_values(['Perc_Contribution'], ascending=[0]).head(1)], 
                                            axis=0)


In [168]:
#Find the most representative document for each topic
# Reset Index    
sent_topics_sorteddf_mallet.reset_index(drop=True, inplace=True)

# Format
sent_topics_sorteddf_mallet.columns = ['Text id','Text title','Topic_Num', 'Topic_Perc_Contrib', 'Topic Keywords' ]
sent_topics_sorteddf_mallet = sent_topics_sorteddf_mallet[['Topic_Num', 'Topic_Perc_Contrib', 'Topic Keywords','Text id','Text title']]

# Show
sent_topics_sorteddf_mallet

,Topic_Num,Topic_Perc_Contrib,Topic Keywords,Text id,Text title
0,0,0.5951,"activities, activity, market, product, product...",133,Capital innovation expenditures
1,1,0.5631,"force, nitrogen, parties, condition, fruit, ag...",119,Greenhouse gas (GHG)
2,2,0.5355,"output, short, environmental, trade, changes, ...",58,Agri-environmental indicators (AEI)
3,3,0.7068,"statistics, level, statistical, international,...",1699,Urban-rural typology
4,4,0.4864,"spent, treatment, company, month, monthly, cre...",15,Credit or store card unclear balance
5,5,0.6679,"protection, property, release, power, employee...",11,European Union (EU)
6,6,0.4089,"mainly, housing, consumer, building, large, or...",146,Retail trade
7,7,0.7964,"european, member, countries, states, union, ec...",10,European Free Trade Association (EFTA)
8,8,0.7691,"number, average, death, values, total, point, ...",163,Life expectancy
9,9,0.4809,"education, sector, public, groups, electricity...",24,Built-up land


### 7. The topics as a mix of keywords.
***

The above LDA model is built with 20 different topics where each topic is a combination of keywords and each keyword contributes with a certain weight to the topic.
Finally,we want to understand the volume and distribution of topics in order to judge how widely it was discussed,so we define the _df_dominant_topics_.


In [169]:
#Print the Keyword in the 20 topics

from pprint import pprint
pprint(lda_model.print_topics())
doc_lda = lda_model[corpus]

[(0,
  '0.106*"activities" + 0.089*"activity" + 0.076*"market" + 0.070*"product" + '
  '0.062*"products" + 0.059*"innovation" + 0.040*"process" + '
  '0.027*"classification" + 0.025*"production" + 0.023*"categories"'),
 (1,
  '0.081*"force" + 0.045*"nitrogen" + 0.031*"parties" + 0.031*"condition" + '
  '0.029*"fruit" + 0.028*"agreement" + 0.022*"nurseries" + 0.020*"young" + '
  '0.016*"accident" + 0.016*"gases"'),
 (2,
  '0.079*"output" + 0.047*"short" + 0.044*"environmental" + 0.042*"trade" + '
  '0.039*"changes" + 0.039*"indicators" + 0.033*"indicator" + 0.029*"measure" '
  '+ 0.025*"secondary" + 0.023*"producers"'),
 (3,
  '0.042*"statistics" + 0.042*"level" + 0.036*"statistical" + '
  '0.030*"international" + 0.029*"country" + 0.028*"defined" + 0.027*"based" + '
  '0.027*"population" + 0.024*"regions" + 0.024*"national"'),
 (4,
  '0.060*"spent" + 0.057*"treatment" + 0.045*"company" + 0.043*"month" + '
  '0.040*"monthly" + 0.039*"credit" + 0.033*"issued" + 0.026*"money" + '
  '0.021

### 8. Visualization of the topics.
***

To visualize the fitted LDA model we use the _pyLDAvis_ package. This is the Python porting of the R package _LDAvis_, see [LDAvis vignette](https://cran.r-project.org/web/packages/LDAvis/vignettes/details.pdf) for details and Chuang, Jason, Manning, Christopher D., and Heer, Jeffrey (2012). Termite: Visualization Techniques for Assessing Textual Topic Models, *Advanced Visual Interfaces* for the theory behind the visualization algorithm. The paper is available [here](https://dl.acm.org/doi/pdf/10.1145/2254556.2254572?casa_token=q2BavKP415QAAAAA:MhcYHzz4PJpC7dNkkm12GL-ohQRUXBgumPJ9l1t_5n3M4qVE1kdDqKGfPmtnR7qbale_ukS-2nJs).



In [170]:
#Visualize the topics

pyLDAvis.enable_notebook()
vis = pyLDAvis.gensim_models.prepare(lda_model, corpus, id2word)
vis

PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
3     -0.226390 -0.080025       1        1  12.857074
11    -0.187022 -0.248189       2        1  10.646288
13    -0.145966  0.050590       3        1   9.788812
7     -0.087717  0.133128       4        1   9.048594
10    -0.142750  0.232981       5        1   8.212530
19    -0.155625  0.062444       6        1   7.732242
14    -0.129263 -0.102202       7        1   6.017547
18    -0.004603 -0.072207       8        1   5.000871
16     0.028831  0.078366       9        1   4.353044
2      0.049498  0.061877      10        1   3.957674
5      0.114300 -0.018436      11        1   3.728963
8      0.031210 -0.131178      12        1   3.421501
9      0.055606 -0.023241      13        1   3.343607
17     0.058718  0.005677      14        1   3.042505
0     -0.010615  0.136611      15        1   2.885836
6      0.137894 -0.027806      16        1   1.761423
1      0.146802 -0.019943      17        1   1.358123
15     0.154138 -0.023173      18        1   1.134404
12     0.153349 -0.003424      19        1   0.877754
4      0.159605 -0.011851      20        1   0.831208, topic_info=           Term        Freq       Total Category  logprob  loglift
14     european  625.000000  625.000000  Default  30.0000  30.0000
236   financial  487.000000  487.000000  Default  29.0000  29.0000
279       goods  469.000000  469.000000  Default  28.0000  28.0000
497      number  341.000000  341.000000  Default  27.0000  27.0000
374    services  457.000000  457.000000  Default  26.0000  26.0000
...         ...         ...         ...      ...      ...      ...
2120     permit    0.039489    1.282336  Topic20  -9.5551   1.3096
3937    credits    0.039483    1.282247  Topic20  -9.5553   1.3095
282    interest    0.812368   27.176678  Topic20  -6.5312   1.2799
2383      index    0.039490    1.282827  Topic20  -9.5551   1.3093
247      months    0.040026   51.237189  Topic20  -9.5416  -2.3647

[817 rows x 6 columns], token_table=      Topic      Freq         Term
term                              
89        1  0.207083  abbreviated
89        4  0.515753  abbreviated
89        5  0.121124  abbreviated
89       10  0.148474  abbreviated
1645     18  0.772945      ability
...     ...       ...          ...
153       7  0.104285        world
668       2  0.317218        years
668       3  0.241361        years
668      12  0.441346        years
779      17  0.934297        young

[1001 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[4, 12, 14, 8, 11, 20, 15, 19, 17, 3, 6, 9, 10, 18, 1, 7, 2, 16, 13, 5])

In [171]:
#Topic distribution across documents

# Number of Documents for Each Topic
topic_counts = df_topic_sents_keywords['Dominant_Topic'].value_counts()

# Percentage of Documents for Each Topic
topic_contribution = round(topic_counts/topic_counts.sum(), 4)

# Topic Number and Keywords
topic_num_keywords = df_topic_sents_keywords[['Dominant_Topic', 'Topic_Keywords']]

# Concatenate Column wise
df_dominant_topics = pd.concat([topic_num_keywords, topic_counts, topic_contribution], axis=1)

# Change Column names
df_dominant_topics.columns = ['Dominant_Topic', 'Topic_Keywords', 'Num_Documents', 'Perc_Documents']

# Show
df_dominant_topics

,Dominant_Topic,Topic_Keywords,Num_Documents,Perc_Documents
0,17,"employed, place, stock, private, similar, acco...",31.0,0.0241
1,17,"employed, place, stock, private, similar, acco...",12.0,0.0093
2,17,"employed, place, stock, private, similar, acco...",35.0,0.0273
3,17,"employed, place, stock, private, similar, acco...",246.0,0.1916
4,7,"european, member, countries, states, union, ec...",3.0,0.0023
...,...,...,...,...
1279,3,"statistics, level, statistical, international,...",NaN,NaN
1280,3,"statistics, level, statistical, international,...",NaN,NaN
1281,10,"activities, water, households, energy, prices,...",NaN,NaN
1282,0,"activities, activity, market, product, product...",NaN,NaN


### 9. The model's coherence score.
***

This is based on the work in Röder, M., Both, A., & Hinneburg, A. (2015, February). Exploring the space of topic coherence measures. In *Proceedings of the eighth ACM international conference on Web search and data mining* (pp. 399-408), available [here](http://svn.aksw.org/papers/2015/WSDM_Topic_Evaluation/public.pdf).

In [172]:
from gensim.models import CoherenceModel

coherence_model_lda = CoherenceModel(model=lda_model, texts=texts, dictionary=id2word, coherence='c_v')
coherence_lda = coherence_model_lda.get_coherence()
print('\nCoherence Score: ', coherence_lda)


Coherence Score:  0.4240461305157298


In [173]:
docs_topics = lda_model.get_document_topics(corpus,minimum_probability=0.3)
res  = [d for d in docs_topics]
res
len(res)

1284